# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# Complete pipeline

## Data Preprocessing

In [ ]:
# Data Loading & Preprocessing
df = pd.read_csv('./data/model_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Define input and output features (excluding 'date' for model training)
input_features = [
    'precipitation + irrigation (mm)',
    'potential evapotranspiration (mm)',
    # 'year',
    # 'month',
    # 'day',
    # 'day_of_week',
    # 'day_of_year',
    'precip_pet_diff',
    'precip_7d_avg',
    'pet_7d_avg',
    # 'precip_7d_sum',
    # 'pet_7d_sum'
]

output_features = [
    'depth 10cm',
    'depth 30cm',
    'depth 60cm',
    'depth 90cm',
    'actual evapotranspiration (mm)',
    'groundwater recharge (mm)'
]

# Convert DataFrame columns to NumPy arrays
X = df[input_features].values
y = df[output_features].values

# Scale the features for better LSTM performance
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Create Sequences for LSTM Input
def create_sequences(X, y, lookback=7):
    """
    Create sequences of length 'lookback' from the time series data.
    For each sample, the input is the previous 'lookback' days
    and the output is the target for the day following the sequence.
    """
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

lookback = 7  # Use the past 7 days to predict the next day
X_seq, y_seq = create_sequences(X_scaled, y_scaled, lookback)

## Split

In [ ]:
# Train/Test Split (Time-based)
split_idx = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

## Model Configuration

In [ ]:

# Build the LSTM Model
model = Sequential()
# First LSTM layer; returns sequences for the next LSTM layer
model.add(LSTM(50, activation='relu', return_sequences=True, 
               input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
# Second LSTM layer; does not return sequences
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
# Dense layer for multi-output regression (6 target variables)
model.add(Dense(len(output_features)))

# Set a custom learning rate
learning_rate = 0.00005  # Try values like 0.0001, 0.0005, 0.005, 0.01, etc.
optimizer = Adam(learning_rate=learning_rate)

# Compile model with custom learning rate
model.compile(optimizer=optimizer, loss='mse')
model.summary()

## Training

In [ ]:
# Train the Model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, 
                    validation_split=0.1, verbose=1)

## Evaluation

In [ ]:
# Predict and Evaluate
# Predict on the test set
y_pred_scaled = model.predict(X_test)
# Inverse transform to get predictions in the original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_inv = scaler_y.inverse_transform(y_test)

# Evaluate using Mean Squared Error (MSE)
mse = mean_squared_error(y_test_inv, y_pred)
print("Test MSE:", mse) 

In [ ]:
# plot training history
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the actual vs. predicted values for each target variable
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.plot(y_test_inv[:, i], label='Actual', color='blue', linewidth=2)
    ax.plot(y_pred[:, i], label='Predicted', color='red', linewidth=2)
    ax.set_title(output_features[i])
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Calculate MSE for each target variable
mse_per_target = {}
for i, feature in enumerate(output_features):
    mse_per_target[feature] = mean_squared_error(y_test_inv[:, i], y_pred[:, i])
    # print(f"{feature}: {mse_per_target[feature]:.2f}")


print("Mean Squared Error for each target variable:")
for feature, mse_val in mse_per_target.items():
    print(f"   {feature}: {mse_val:.2f}")
print("\nOverall Mean Squared Error:", mse)